<a href="https://colab.research.google.com/github/Team07Genai/Team7GenAI/blob/main/Team_7_assessment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pdfplumber

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

import fitz
from io import BytesIO

pdf_file = next(iter(uploaded.values()))
pdf_document = fitz.open(stream=BytesIO(pdf_file), filetype="pdf")

print(len(pdf_document))
text = ''
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    text += page.get_text()

with open('/content/extracted_text.txt', 'w') as file:
    file.write(text)

print(text[:1000])

Saving Paris2024-QS-Athletics.pdf to Paris2024-QS-Athletics (1).pdf
11


In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import openai

from google.colab import userdata
OPENAI_API_KEY = userdata.get('API_Key')
import openai
openai.api_key = OPENAI_API_KEY

client = openai.OpenAI(
   api_key = openai.api_key,
)

def answer_question(question, context):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=f"Context: {context}\nQuestion: {question}\nAnswer:"    )
    return response.choices[0].text.strip()

with open('extracted_text.txt', 'r') as file:
    context = file.read()

questions = [
    "What are the eligibility rules for athletes?",
    "What are the entry standards for the marathon?",
    "How can athletes qualify for the relay events?"
]

for question in questions:
    answer = answer_question(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering")

def answer_question_local(question, context):
    result = qa_pipeline(question=question, context=context)
    return result['answer']

with open('extracted_text.txt', 'r') as file:
    context = file.read()

questions = [
     "Give list of all Men's Events"
]

for question in questions:
    answer = answer_question_local(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Question: Give list of all Men's Events
Answer: For 10,000m



In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Correct model identifier (replace with the actual LLaMA model identifier)
model_name = "facebook/opt-1.3b"  # Example model, replace with correct LLaMA identifier

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Create a QA pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

def split_context(context, max_length=1000):
    """Split the context into chunks of a maximum length."""
    tokens = tokenizer.encode(context, truncation=False)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return [tokenizer.decode(chunk) for chunk in chunks]

def answer_question_local(question, context):
    """Answer a question based on the given context."""
    chunks = split_context(context)
    answers = []
    for chunk in chunks:
        result = qa_pipeline(question=question, context=chunk)
        answers.append(result['answer'])

    # Combine answers from all chunks
    combined_answer = ' '.join(answers)
    return combined_answer

# Load the extracted text
with open('extracted_text.txt', 'r') as file:
    context = file.read()

# Example questions
questions = [
    "What are the eligibility rules for athletes?",
    "What are the entry standards for the marathon?",
    "How can athletes qualify for the relay events?",
    "How many mixed events are there ?"
]

# Get answers for each question
for question in questions:
    answer = answer_question_local(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

Some weights of OPTForQuestionAnswering were not initialized from the model checkpoint at facebook/opt-1.3b and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: What are the eligibility rules for athletes?
Answer: )
 QUOTAPLACES  PLACESF.
 REALLOCATION OF UNUSED 0085.50Javelin

Question: What are the entry standards for the marathon?
Answer: )
 QUOTAPLACES  Area Race Walking Judges must be on duty.
o Track performances ( 0085.50Javelin



KeyboardInterrupt: 